In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from keras import models
from gtsrb_loader.get_folderpath import get_folderpath
from gtsrb_loader.load_data import load_bounding_boxes_generator

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.patches as patches

# This is a bit of magic to make matplotlib figures appear inline in the notebook
# rather than in a new window.
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# Some more magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [ ]:
model = models.load_model(os.path.join('model_files', 'localization', '25-07-17-localization.h5'))

In [ ]:
#path = get_folderpath(original_images=False, subset='test')
path = get_folderpath(original_images=False, subset='train')
path

In [ ]:
gen = load_bounding_boxes_generator(batch_size=30, path=path)

In [ ]:
pics_list, bboxes_list = next(gen)
bboxes_list = np.array(bboxes_list, dtype=int)

In [ ]:
predictions = np.rint(model.predict(np.array(pics_list), verbose=1)).astype(int)

In [ ]:
x, y = bboxes_list[:,0], bboxes_list[:,1]
width = bboxes_list[:,2] - bboxes_list[:,0]
heigth = bboxes_list[:,3] - bboxes_list[:,1] 

In [ ]:
x_pred, y_pred = predictions[:,0], predictions[:,1]
width_pred = predictions[:,2] - predictions[:,0]
heigth_pred = predictions[:,3] - predictions[:,1]

In [ ]:
for i, image in enumerate(pics_list):
    ax = plt.subplot(5, 6, i +1)
    plt.imshow(image)
    plt.axis('off')
    # correct bounding boxes in yellow
    rect = patches.Rectangle((x[i], y[i]), width[i], heigth[i], linewidth=1, edgecolor='y', facecolor='none')
    ax.add_patch(rect)
    # predicted bounding boxes in red
    rect = patches.Rectangle((x_pred[i], y_pred[i]), width_pred[i], heigth_pred[i], linewidth=1, edgecolor='r', facecolor='none')
    ax.add_patch(rect)
plt.show()

In [ ]:
error = np.linalg.norm(predictions - bboxes_list, axis=1).reshape((30, 1))
pd.DataFrame(np.concatenate((predictions, bboxes_list, error), axis=1),
             columns=['x1', 'y1', 'x2', 'y2', 'x1 pred', 'y1 pred', 'x2 pred', 'y2 pred', 'l2 error'])